In [1]:
import os
import pandas as pd

In [2]:
DATADIR = os.path.join(os.getenv("DATA_DIR"), 'processed_journey')
DATADIR

'/Volumes/GoogleDrive/Team Drives/GOV.UK teams/2018-2019/Q3/Knowledge up Q3/Data science/data/processed_journey'

In [3]:
journeys = pd.read_csv(os.path.join(DATADIR, "doo_prelim_meta_standard_with_pageseq_from_29-10_to_03-11-2018.csv.gz"), compression='gzip')

In [5]:
journeys.shape

(1722287, 16)

In [4]:
journeys.head(10)

,Sequence,PageSequence,Occurrences,Page_Seq_Occurrences,DeviceCategories,Dates,Page_Event_List,Page_List,PageSequence_internal,Event_List,num_event_cats,Event_cats_agg,Event_cat_act_agg,Page_List_NL,Page_Seq_NL,Occurrences_NL
0,/guidance/rates-and-thresholds-for-employers-2...,/guidance/rates-and-thresholds-for-employers-2...,77,284.0,"[('desktop', 59), ('mobile', 9), ('tablet', 9)]","[('20181031', 33), ('20181029', 27), ('2018110...",[('/guidance/rates-and-thresholds-for-employer...,['/guidance/rates-and-thresholds-for-employers...,/guidance/rates-and-thresholds-for-employers-2...,"[('PAGE_NULL', 'PAGE_NULL')]",1,"[('PAGE_NULL', 1)]","[(('PAGE_NULL', 'PAGE_NULL'), 1)]",['/guidance/rates-and-thresholds-for-employers...,/guidance/rates-and-thresholds-for-employers-2...,4930.0
1,/sure-start-maternity-grant/how-to-claim<<PAGE...,/sure-start-maternity-grant/how-to-claim>>/gov...,89,331.0,"[('mobile', 43), ('desktop', 43), ('tablet', 3)]","[('20181031', 34), ('20181029', 31), ('2018110...","[('/sure-start-maternity-grant/how-to-claim', ...","['/sure-start-maternity-grant/how-to-claim', '...",/sure-start-maternity-grant/how-to-claim>>/gov...,"[('PAGE_NULL', 'PAGE_NULL'), ('PAGE_NULL', 'PA...",3,"[('PAGE_NULL', 2), ('External Link Clicked', 1...","[(('PAGE_NULL', 'PAGE_NULL'), 2), (('External ...","['/sure-start-maternity-grant/how-to-claim', '...",/sure-start-maternity-grant/how-to-claim>>/gov...,5408.0
2,/historic-vehicles/apply-for-vehicle-tax-exemp...,/historic-vehicles/apply-for-vehicle-tax-exemp...,96,306.0,"[('mobile', 56), ('desktop', 30), ('tablet', 10)]","[('20181031', 35), ('20181029', 25), ('2018110...",[('/historic-vehicles/apply-for-vehicle-tax-ex...,['/historic-vehicles/apply-for-vehicle-tax-exe...,/historic-vehicles/apply-for-vehicle-tax-exemp...,"[('PAGE_NULL', 'PAGE_NULL')]",1,"[('PAGE_NULL', 1)]","[(('PAGE_NULL', 'PAGE_NULL'), 1)]",['/historic-vehicles/apply-for-vehicle-tax-exe...,/historic-vehicles/apply-for-vehicle-tax-exemp...,1536.0
3,/government-gateway<<PAGE<:<NULL<:<NULL,/government-gateway,1138,8909.0,"[('mobile', 463), ('tablet', 119), ('desktop',...","[('20181031', 376), ('20181029', 413), ('20181...","[('/government-gateway', 'PAGE<:<NULL<:<NULL')]",['/government-gateway'],/government-gateway,"[('PAGE_NULL', 'PAGE_NULL')]",1,"[('PAGE_NULL', 1)]","[(('PAGE_NULL', 'PAGE_NULL'), 1)]",['/government-gateway'],/government-gateway,628989.0
4,/guidance/passport-rules-for-travel-to-europe-...,/guidance/passport-rules-for-travel-to-europe-...,57,11214.0,"[('desktop', 47), ('tablet', 10)]","[('20181031', 33), ('20181029', 6), ('20181102...",[('/guidance/passport-rules-for-travel-to-euro...,['/guidance/passport-rules-for-travel-to-europ...,/guidance/passport-rules-for-travel-to-europe-...,"[('user_satisfaction_survey', 'banner_shown'),...",2,"[('user_satisfaction_survey', 1), ('PAGE_NULL'...","[(('user_satisfaction_survey', 'banner_shown')...",['/guidance/passport-rules-for-travel-to-europ...,/guidance/passport-rules-for-travel-to-europe-...,550475.0
5,/claim-national-insurance-refund<<PAGE<:<NULL<...,/claim-national-insurance-refund,143,437.0,"[('desktop', 50), ('mobile', 88), ('tablet', 5)]","[('20181031', 35), ('20181029', 65), ('2018110...","[('/claim-national-insurance-refund', 'PAGE<:<...",['/claim-national-insurance-refund'],/claim-national-insurance-refund,"[('PAGE_NULL', 'PAGE_NULL')]",1,"[('PAGE_NULL', 1)]","[(('PAGE_NULL', 'PAGE_NULL'), 1)]",['/claim-national-insurance-refund'],/claim-national-insurance-refund,1770.0
6,/government/collections/data-protection-act-20...,/government/collections/data-protection-act-20...,94,531.0,"[('mobile', 12), ('desktop', 80), ('tablet', 2)]","[('20181031', 33), ('20181029', 39), ('2018110...",[('/government/collections/data-protection-act...,['/government/collections/data-protection-act-...,/government/collections/data-protection-act-20...,"[('PAGE_NULL', 'PAGE_NULL'), ('navDocumentColl...",3,"[('PAGE_NULL', 2), ('navDocumentCollectionLink...","[(('PAGE_NULL', 'PAGE_NULL

In [6]:
def contains_search(Page_List):
    search_in_sequence = 0
    thelist  = pd.eval(Page_List)
    
    for page in thelist:
        
        if ('/search?' in page) or ('/search/' in page):
            search_in_sequence = 1
            print(page)
    return search_in_sequence

def count_search(Page_List):
    times_search_in_sequence = 0
    thelist  = pd.eval(Page_List)
    
    for page in thelist:
        if ('/search?' in page) or ('/search/' in page):
            times_search_in_sequence += 1
            print(page)
    return times_search_in_sequence


journeys['contains_search'] = list(map(contains_search, journeys['Page_List']))

journeys['count_search'] = list(map(count_search, journeys['Page_List']))

/search?q=passport
/search?q=income+tax
/search?q=ilr
/search?q=mynewham.gov.uk
/search?q=mynewham.gov.uk
/search?q=[postcode]
/search?q=mr+gs+eat+&+shake=
/search?q=notice+143
/search?q=stamp+duty
/search?q=self+assessment+
/search?q=apprenticeship
/search?q=flight
/search?q=deed+poll
/search?q=universal+credit
/search?q=blue+badge
/search?q=
/search?q=tax+rebate
/search?q=self+employed+
/search?q=child+benefit
/search?q=tax+free+childcare
/search?q=mot+tax+checker
/search?q=provisional+
/search?q=
/search/advanced?group=services&topic=/welfare/universal-credit
/search?q=theory
/search?q=minimum+wage+
/search?q=road+tax
/search?q=p85
/search?q=dart+charge
/search?q=theory+
/search?q=childcare+account
/search?q=driving+test+
/search?q=winter+fuel+allowance
/search?q=esta
/search?q=driving+test+
/search?q=change+address+driving+licence
/search?q=tax+free+childcare
/search?q=track+dbs
/search?q=cold+weather+payments
/search?q=national+insurance+number+
/search?q=pension+credits
/search?q

/search?q=sorn
/search?q=childcare+account
/search?q=change+practical+
/search?q=personal+tax
/search?q=n1&filter_organisations[]=hm-courts-and-tribunals-service
/search?q=30+hours
/search?q=jobs
/search?q=www.gov.uk/vehicle-tax
/search?q=tax+free+child
/search?q=self+assessment
/search?q=register+to+vote
/search?q=self+assessment+forms+and+helpsheets
/search?q=school+leaving+age
/search?q=tv+licence
/search?q=notice+725
/search?q=childcare+account
/search?q=childcare+account
/search?q=part+time
/search?q=car+tax
/search?q=share+my+details
/search?q=visa+fees
/search?q=childcare+account
/search?q=childcare
/search?q=hmrc
/search?q=pip
/search?q=childcare+account
/search?q=find+apprenticeship
/search?q=sic+code
/search?q=tax
/search?q=power+of+arrest&filter_organisations[]=hm-courts-and-tribunals-service
/search?q=rdr3
/search?q=sa100
/search?q=dart
/search?q=tax+credits
/search?q=national+insurance+record
/search?q=driving+licence+check
/search?q=minimum+wage
/search?q=c1a&filter_organ

/search?q=change+of+address
/search?q=login
/search?q=part+time
/search?q=export+foods
/search?q=is+my+car+taxed
/search?q=income+tax
/search?q=personal+tax+account
/search?q=childcare+account
/search?q=oversea+passport
/search?q=v149/1
/search?q=universil+credit
/search?q=benefit+fraud
/search?q=cf411a
/search?q=continuation+sheet
/search?q=renew+driving+licence
/search?q=universal+credit
/search?q=701/7
/search?q=universal+c
/search?q=book+a+driving+test
/search?q=childcare+services
/search?q=private+plates
/search?q=v5c
/search?q=road+adoption
/search?q=gateway
/search?q=lost
/search?q=prescriptions
/search?q=childcare+account
/search?q=tax+free+childcare
/search?q=beis
/search?q=my+licence
/search?q=fl415&filter_organisations[]=hm-courts-and-tribunals-service
/search?q=pay+road+tax
/search?q=mot+testing+service
/search?q=national+insurance
/search?q=pay+corporation+tax
/search?q=iht406
/search?q=check+tax
/search?q=probate+registry
/search?q=driving+license+check
/search?q=tax+free

/search?q=tax
/search?q=vehicle+tax
/search?q=sorn
/search?q=universal+credit
/search?q=contact
/search?q=paternity+leave
/search?q=universal+credit
/search?q=mot
/search?q=new+zealand
/search?q=ofsted
/search?q=n180&filter_organisations[]=hm-courts-and-tribunals-service
/search?q=n180
/search?q=n180&filter_organisations[]=hm-courts-and-tribunals-service
/search?q=childcare+account
/search?q=universalcredit
/search?o=universalcredit&q=universal+credit
/search?q=res1
/search?q=carers+allowance+
/search?q=vehicle+tax
/search?q=sorn
/search?q=foreign+travel
/search?q=universal+credit
/search?q=car+tax
/search?q=carers+allowance
/search?q=universal+credit
/search?q=blue+badge
/search?q=tax+return
/search?q=road+tax
/search?q=manage+your+tax+credits
/search?q=hmrc
/search?q=crmr1
/search?q=inf188/6
/search?q=jobseekers
/search?q=road+tax
/search?q=login
/search?q=sc2
/search?q=djp
/search?q=vehicle+tax
/search?q=tax+a+vehicle
/search?q=login
/search?q=mot
/search?q=64-8
/search?q=vehicle+ta

/search?q=road+tax
/search?q=road+tax
/search?q=road+tax
/search?q=nino
/search?q=log+in+to+universal+credit+
/search?q=sign+in
/search?q=prison+visit
/search?q=student+finance
/search?q=personal+tac+account
/search?q=car+tax
/search?q=apprenticeship+
/search?q=sorn
/search?q=journal
/search?q=entry+requirements
/search?q=road+tax
/search?q=fraud
/search?q=cwf1
/search?q=vat
/search?q=universal+credit
/search?q=m5
/search?q=change+address+on+v5
/search?q=r40
/search?q=sorn
/search?q=sa100
/search?q=car+tax
/search?q=right+to+work
/search?q=national+minimum+wage
/search?q=ex107&filter_organisations[]=hm-courts-and-tribunals-service
/search?q=companies+house
/search?q=n180&filter_organisations[]=hm-courts-and-tribunals-service
/search?q=personal+tax+account
/search?q=tax+my+car
/search?q=universal+credit
/search?q=p800
/search?q=sign+in
/search?q=petitions
/search?q=sign+in
/search?q=universal+credit+login+
/search?q=vehicle+tax+
/search?q=uniform+tax
/search?q=tax+a+car
/search?q=car+ta

...

/search?q=ico
/search?q=learn+to+drive
/search?q=naturalisation
/search?q=windrush
/search?q=mr01
/search?q=hmrc+phishing
/search?q=new+style+esa
/search?q=sdlt
/search?q=certificate+of+residence
/search?q=lpa+forms
/search?q=london+diplomatic+list
/search?q=practice+guides
/search?q=state+pension+forecast
/search?q=cog1
/search?q=vat+online
/search?q=state+pension
/search?q=charging+scheme
/search?q=rle1
/search?q=vat
/search?q=v5c
/search?q=statutory+sick+pay
/search?q=venice
/search?q=new+style+jobseekers+allowance
/search?q=mot+history
/search?q=check+code
/search?q=insurance
/search?q=n161&filter_organisations[]=hm-courts-and-tribunals-service
/search?q=working+tax+credits
/search?q=cold+weather+payments
/search?q=mot
/search?q=mot+check
/search?q=new+starter+checklist
/search?q=pension
/search?q=basic+dbs
/search?q=lp12
/search?q=legalisation+office
/search?q=mot+history
/search?q=childcare+account
/search?filter_manual[]=/guidance/childminders-and-childcare-providers-register-wi

/search?q=universal+credit+log+in
/search?q=view+driving+licence
/search?q=high+income+charge
/search?q=tell+us+once
/search?q=splenectomy
/search?q=birth+certificate
/search?q=personal+tax+account
/search?q=esa
/search?q=sa103
/search?q=dvla
/search?q=phishing
/search?q=tax+return
/search?q=redundancy+
/search?q=phishing
/search?q=calculators
/search?q=companies+house
/search?q=universalcredit
/search?o=universalcredit&q=universal+credit
/search?q=rates+and+allowances+for+travel
/search?q=income+support
/search?q=jsa
/search?q=change+address
/search?q=pip+rates+2018
/search?q=ipc+br1
/search?q=mot
/search?q=problems+signing+in
/search?q=700/45
/search?q=ap01
/search?q=advanced+learner+loan
/search?q=uniform+tax
/search?q=universal+credit
/search?q=personal+
/search?q=mot+history
/search?q=childcare+account
/search?q=v112
/search?q=dla
/search?q=car+tax
/search?q=login
/search?q=login+childcare
/search?q=tax
/search?q=renew+passport
/search?q=assign+a+number+plate
/search?q=share+cod
/

/search?q=check+mot
/search?q=childcare+tax+free
/search?q=practical+test
/search?q=driving+test
/search?q=car+seats
/search?q=land+registry
/search?q=childcare+account
/search?q=c100&filter_organisations[]=hm-courts-and-tribunals-service
/search?q=c100&filter_organisations[]=hm-courts-and-tribunals-service
/search?q=childcare+account
/search?q=tax+free+child+care
/search?q=mot+check
/search?q=allan
/search?q=fees
/search?q=tax+credits
/search?q=book+theory+
/search?q=flr+m
/search?q=childcare+account
/search?q=childcare+account
/search?q=mot+history+
/search?q=manage+tax+credits+
/search?q=childcare+account
/search?q=childcare+account
/search?q=child+care+account
/search?q=tax+free+childcare+
/search?q=driving+test
/search?q=driving+test
/search?q=book+test
/search?q=mot
/search?q=childcare+account
/search?q=childcare+account
/search?q=tax
/search?q=child+benefit
/search?q=mot+history+
/search?q=register+to+vote
/search?q=childcare+account
/search?q=birth+certificate
/search?q=driving

/search?q=mot+history
/search?q=
/search?q=benefit+fraud
/search?q=online+services
/search?q=finding+a+job
/search?q=find+a+jobs
/search?q=universal+credit+login
/search?q=ca3837
/search?q=30+hours
/search?q=p85
/search?q=winter+fuel+payment
/search?q=turkey
/search?q=national+minimum+wage
/search?q=paying+hmrc
/search?q=sa106
/search?q=book+a+test
/search?q=manage+your+tax+credits
/search?q=childcare+account
/search?q=attendance+allowance
/search?q=[email]
/search?q=log+to+universal+credit
/search?q=childcare+account
/search?q=passport
/search?q=ch2
/search?q=tax+free+child+care
/search?q=manage+tax+credits
/search?q=register+to+vote
/search?q=change+address
/search?q=childcare+account
/search?q=student+loan
/search?q=theory+test
/search?q=lost+provisional
/search?q=mot
/search?q=mot+check
/search?q=life+in+uk
/search?q=tax+car
/search?q=sms
/search?q=uniform
/search?q=find+a+job
/search?q=vote
/search?q=paye
/search?q=register+to+vote
/search?q=company+car
/search?q=pay+a+prisoner
/s

/search?q=mot+history
/search?q=mot+history
/search?q=tax+free+childcare
/search?q=dla
/search?q=tell+dvla
/search?q=sold+car
/search?q=ch2
/search?q=plates
/search?q=universal+credit
/search?q=universal+credit
/search?q=tax+code
/search?q=tax+code
/search?q=holiday+entitlement
/search?q=hmrc
/search?q=tell+dvla
/search?q=universal+credit
/search?q=universal+credit
/search?q=universal+credit
/search?q=notice+143
/search?q=universal+credit
/search?q=ap1
/search?q=selling+a+car
/search?q=tax+code
/search?q=hmrc
/search?q=hmrc
/search?q=p53z
/search?q=p53z
/search?q=hmrc+phishing
/search?q=universal+credit
/search?q=mandatory+reconsideration
/search?q=tax
/search?q=sold+vehicle
/search?q=pta
/search?q=pta
/search?q=universal+credit
/search?q=national+insurance+number
/search?q=ofsted
/search?q=ofsted
/search?q=pip
/search?q=ds01
/search?q=power+of+attorney
/search?q=universal+credit
/search?q=universal+credit
/search?q=tax+rebate
/search?q=universal+credit+sign+in+
/search?q=student+finan

In [8]:
journeys['contains_search_n'] = journeys['contains_search']*journeys['Page_Seq_Occurrences']
journeys['contains_search_n']



0          0.0
1          0.0
2          0.0
3          0.0
4          0.0
5          0.0
6          0.0
7          0.0
8          0.0
9          0.0
10         0.0
11         0.0
12         0.0
13         0.0
14         0.0
15         0.0
16         0.0
17         0.0
18         0.0
19         0.0
20         0.0
21         0.0
22         0.0
23         0.0
24         0.0
25         0.0
26         0.0
27         0.0
28         0.0
29         0.0
          ... 
1722257    0.0
1722258    0.0
1722259    0.0
1722260    0.0
1722261    0.0
1722262    0.0
1722263    0.0
1722264    0.0
1722265    0.0
1722266    0.0
1722267    0.0
1722268    0.0
1722269    0.0
1722270    0.0
1722271    0.0
1722272    0.0
1722273    0.0
1722274    0.0
1722275    0.0
1722276    0.0
1722277    0.0
1722278    0.0
1722279    0.0
1722280    0.0
1722281    0.0
1722282    0.0
1722283    0.0
1722284    0.0
1722285    0.0
1722286    0.0
Name: contains_search_n, Length: 1722287, dtype: float64

In [9]:
print("{}/{} ({}%) of journeys contain internal search in their path".format(journeys['contains_search_n'].sum(),
                                                                             journeys['Page_Seq_Occurrences'].sum(),
                                                                             journeys['contains_search_n'].sum()/journeys['Page_Seq_Occurrences'].sum()*100))

4377196.0/67924193765.0 (0.006444236960903735%) of journeys contain internal search in their path


In [10]:
journeys['contains_search'].value_counts()

0    1686856
1      35431
Name: contains_search, dtype: int64

In [11]:
journeys['contains_search'].value_counts(normalize=True)

0    0.979428
1    0.020572
Name: contains_search, dtype: float64

In [104]:
journeys['count_search'].describe()

count    606735.000000
mean          0.016203
std           0.131750
min           0.000000
25%           0.000000
50%           0.000000
75%           0.000000
max           4.000000
Name: count_search, dtype: float64

In [ ]:
def df['length'] = len

In [28]:
df1 = df[df['Occurrences']<5]

In [33]:
df1

,Sequence,PageSequence,Occurrences,Page_Seq_Occurrences,DeviceCategories,Dates,Page_Event_List,Page_List,PageSequence_internal,Event_List,num_event_cats,Event_cats_agg,Event_cat_act_agg,Page_List_NL,Page_Seq_NL,Occurrences_NL
25,/government/organisations/nuclear-decommission...,/government/organisations/nuclear-decommission...,1,3.0,"[('mobile', 1)]","[('20181018', 1)]",[('/government/organisations/nuclear-decommiss...,['/government/organisations/nuclear-decommissi...,/government/organisations/nuclear-decommission...,"[('PAGE_NULL', 'PAGE_NULL')]",1,"[('PAGE_NULL', 1)]","[(('PAGE_NULL', 'PAGE_NULL'), 1)]",['/government/organisations/nuclear-decommissi...,/government/organisations/nuclear-decommission...,9.0
26,/foreign-travel-advice/new-zealand/travel-advi...,/foreign-travel-advice/new-zealand/travel-advi...,1,2.0,"[('mobile', 1)]","[('20181018', 1)]",[('/foreign-travel-advice/new-zealand/travel-a...,['/foreign-travel-advice/new-zealand/travel-ad...,/foreign-travel-advice/new-zealand/travel-advi...,"[('PAGE_NULL', 'PAGE_NULL')]",1,"[('PAGE_NULL', 1)]","[(('PAGE_NULL', 'PAGE_NULL'), 1)]",['/foreign-travel-advice/new-zealand/travel-ad...,/foreign-travel-advice/new-zealand/travel-advi...,4.0
27,/government/publications/italy-list-of-transla...,/government/publications/italy-list-of-transla...,1,3.0,"[('mobile', 1)]","[('20181018', 1)]",[('/government/publications/italy-list-of-tran...,['/government/publications/italy-list-of-trans...,/government/publications/italy-list-of-transla...,"[('PAGE_NULL', 'PAGE_NULL')]",1,"[('PAGE_NULL', 1)]","[(('PAGE_NULL', 'PAGE_NULL'), 1)]",['/government/publications/italy-list-of-trans...,/government/publications/italy-list-of-transla...,9.0
28,/employment-tribunal-decisions/mrs-p-bhatia-an...,/employment-tribunal-decisions/mrs-p-bhatia-an...,1,2.0,"[('mobile', 1)]","[('20181018', 1)]",[('/employment-tribunal-decisions/mrs-p-bhatia...,['/employment-tribunal-decisions/mrs-p-bhatia-...,/employment-tribunal-decisions/mrs-p-bhatia-an...,"[('PAGE_NULL', 'PAGE_NULL')]",1,"[('PAGE_NULL', 1)]","[(('PAGE_NULL', 'PAGE_NULL'), 1)]",['/employment-tribunal-decisions/mrs-p-bhatia-...,/employment-tribunal-decisions/mrs-p-bhatia-an...,4.0
29,/search?q=xx<<PAGE<:<NULL<:<NULL,/search?q=xx,1,2.0,"[('mobile', 1)]","[('20181018', 1)]","[('/search?q=xx', 'PAGE<:<NULL<:<NULL')]",['/search?q=xx'],/search?q=xx,"[('PAGE_NULL', 'PAGE_NULL')]",1,"[('PAGE_NULL', 1)]","[(('PAGE_NULL', 'PAGE_NULL'), 1)]",['/search?q=xx'],/search?q=xx,4.0
30,/government/publications/teacher-misconduct-pa...,/government/publications/teacher-misconduct-pa...,1,2.0,"[('mobile', 1)]","[('20181018', 1)]",[('/government/publications/teacher-misconduct...,['/government/publications/teacher-misconduct-...,/government/publications/teacher-misconduct-pa...,"[('PAGE_NULL', 'PAGE_NULL')]",1,"[('PAGE_NULL', 1)]","[(('PAGE_NULL', 'PAGE_NULL'), 1)]",['/government/publications/teacher-misconduct-...,/government/publications/teacher-misconduct-pa...,4.0
31,/government/publications/hungary-consular-fees...,/government/publications/hungary-consular-fees,1,2.0,"[('mobile', 1)]","[('20181018', 1)]",[('/government/publications/hungary-consular-f...,['/government/publications/hungary-consular-fe...,/government/publications/hungary-consular-fees,"[('PAGE_NULL', 'PAGE_NULL')]",1,"[('PAGE_NULL', 1)]","[(('PAGE_NULL', 'PAGE_NULL'), 1)]",['/government/publications/hungary-consular-fe...,/government/publications/hungary-consular-fees,4.0
32,/government/publications/childrens-commissione...,/government/publications/childrens-commissione...,1,3.0,"[('mobile', 1)]","[('20181018', 1)]",[('/government/publications/childrens-commissi...,['/government/publications/childrens-commissio...,/government/publications/childrens-commissione...,"[('PAGE_NULL', 'PAGE_NULL')]",1,"[('PAGE_NULL', 1)]","[(('PAGE_NULL', 'PAGE_NULL'), 1)]",['/government/publications/childrens-commissio...,/government/publications/childrens-commissione...,6.0
33,/government/publications/denaturing-of-control...,/go

In [38]:
df1['Sequence'][602379]

'/government/organisations/companies-house<<PAGE<:<NULL<:<NULL>>/government/organisations/companies-house<<EVENT<:<External Link Clicked<:<https://beta.companieshouse.gov.uk/>>/government/organisations/companies-house<<PAGE<:<NULL<:<NULL>>/file-an-annual-return-with-companies-house<<PAGE<:<NULL<:<NULL>>/file-an-annual-return-with-companies-house<<EVENT<:<External Link Clicked<:<https://ewf.companieshouse.gov.uk/>>/government/organisations/companies-house<<PAGE<:<NULL<:<NULL>>/government/organisations/companies-house<<EVENT<:<External Link Clicked<:<https://beta.companieshouse.gov.uk/'